In [1]:
using Flux
using MLDatasets
using Flux.Data: DataLoader

┌ Info: CUDAnative.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAnative /home/wmkouw/.julia/packages/CUDAnative/hfulr/src/CUDAnative.jl:192


In [3]:
# Load data sets
X,Y = MLDatasets.MNIST.traindata(Float32)

# Image to vector
X = reshape(X, 28^2, :)

# Start DataLoader
DataLoader(X,Y, batchsize=32, shuffle=true)

DataLoader((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [5, 0, 4, 1, 9, 2, 1, 3, 1, 4  …  9, 2, 9, 5, 1, 8, 3, 5, 6, 8]), 32, 60000, true, 60000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], true)

In [4]:
function Encoder(x; input_dim=2, hidden_dim=3, latent_dim=1, nonlinearity=tanh)
    "Mapping from observed space to latent space"

    # Map input to hidden layer
    h = Dense(input_dim, hidden_dim, nonlinearity)(x)

    # Map hidden layer activity to mean and log-variance
    μ = Dense(hidden_dim, latent_dim, nonlinearity)(h)
    logσ = Dense(hidden_dim, latent_dim, nonlinearity)(h)

    return μ, logσ
end

Encoder (generic function with 1 method)

In [5]:
function Decoder(z; latent_dim=1, hidden_dim=3, input_dim=2, nonlinearity=tanh)
   "Mapping from latent space to observed space" 
    
    # Latent to hidden
    h = Dense(latent_dim, hidden_dim, nonlinearity)(z)
    
    # Hidden to input
    return Dense(hidden_dim, input_dim, nonlinearity)(h)
end

Decoder (generic function with 1 method)

In [9]:
function loss(X, encoder, decoder; λ=0.1)
    "Loss layer"
    
    # Number of samples
    D,N = size(X)
    
    # Encode samples
    μ, logσ = encoder(X)
    
    # Generate samples in latent space
    z = μ + randn(Float32, (D,N)) .* exp.(logσ)
    
    # Decode generated samples
    x_hat = decoder(z)
   
    # KL-divergence
    KL = 0.5 * sum(@. (exp(2. *logσ) + μ^2 -1. - 2. *logσ)) / N

    # Reconstruction error
    logp_x_z = -sum(logitbinarycrossentropy.(x_hat, x)) / N
    
    # regularization
    reg = λ * sum(x->sum(x.^2), Flux.params(decoder))

    return -logp_x_z + KL + reg
end

loss (generic function with 1 method)